In [1]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
from keras.utils.np_utils import to_categorical

# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [3]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_dataset.csv", sep=',', encoding='utf-8', index=False)


df = describe_dataset("./dataset.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 5151 
Number of columns: 21

Labels: 
C    2685
L    2466
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [4]:
# load dataset
df = pd.read_csv("./dataset.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

print(f'Number of rows: {df.shape[0]} \nNumber of columns: {df.shape[1]}\n')
print(f"Labels: \n{df['label'].value_counts()}\n")

Number of rows: 5151 
Number of columns: 21

Labels: 
0    2685
1    2466
Name: label, dtype: int64



In [5]:
# Standard Scaling of features
sc = StandardScaler()

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.fit_transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

In [7]:
model = Sequential()
model.add(Dense(20, input_dim = 20, activation = "relu"))
model.add(Dense(10, activation = "relu"))
model.add(Dense(2, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                420       
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 652
Trainable params: 652
Non-trainable params: 0
_________________________________________________________________


2022-10-15 00:47:49.440928: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-15 00:47:49.441555: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test))

Epoch 1/100


2022-10-15 00:48:05.540839: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-15 00:48:05.741505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


410/412 [============================>.] - ETA: 0s - loss: 0.0415 - accuracy: 0.9883

2022-10-15 00:48:08.924773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


412/412 [==============================] - 4s 8ms/step - loss: 0.0413 - accuracy: 0.9883 - val_loss: 0.0062 - val_accuracy: 0.9990
Epoch 2/100
412/412 [==============================] - 3s 8ms/step - loss: 0.0233 - accuracy: 0.9939 - val_loss: 0.0046 - val_accuracy: 0.9990
Epoch 3/100
412/412 [==============================] - 3s 7ms/step - loss: 0.0167 - accuracy: 0.9954 - val_loss: 0.0043 - val_accuracy: 0.9990
Epoch 4/100
412/412 [==============================] - 3s 8ms/step - loss: 0.0160 - accuracy: 0.9949 - val_loss: 0.0054 - val_accuracy: 0.9990
Epoch 5/100
412/412 [==============================] - 3s 7ms/step - loss: 0.0188 - accuracy: 0.9949 - val_loss: 0.0073 - val_accuracy: 0.9990
Epoch 6/100
412/412 [==============================] - 3s 7ms/step - loss: 0.0125 - accuracy: 0.9947 - val_loss: 0.0063 - val_accuracy: 0.9981
Epoch 7/100
412/412 [==============================] - 3s 7ms/step - loss: 0.0204 - accuracy: 0.9937 - val_loss: 0.0083 - val_accuracy: 0.9981
Epoch 8/100

In [9]:
from sklearn.metrics import confusion_matrix

predict_x = model.predict(x_test) 
y_pred_class = np.argmax(predict_x, axis=1)

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)

confusion_matrix(y_test_class, y_pred_class)

 1/33 [..............................] - ETA: 0s

2022-10-15 00:57:26.901323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 2ms/step


array([[536,   0],
       [  1, 494]])

In [10]:
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       536
           1       1.00      1.00      1.00       495

    accuracy                           1.00      1031
   macro avg       1.00      1.00      1.00      1031
weighted avg       1.00      1.00      1.00      1031



In [11]:
# Dump the best model to a pickle file
with open("./model/bicep_model_deep_learning.pkl", "wb") as f:
    pickle.dump(model, f)



INFO:tensorflow:Assets written to: ram://c59159f4-5014-40d0-89a4-cc4b78edd148/assets


In [6]:
# Dump input scaler
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)